<a href="https://colab.research.google.com/github/samueleallen/Scraping-Val-Data/blob/main/ValorantMatchPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [223]:
import pandas as pd

In [224]:
matches_df = pd.read_csv("overall_game_stats_fixed_fr_fr.csv")
matches_df.head()

,Player,R2.0,ACS,K,D,A,+/- K/D,KAST,ADR,HS%,FK,FD,+/- FK/FD,date,Team,vs Team,result,final_score
0,Derke,1.23,287.0,20,19,4,1,78%,179.0,35%,8.0,4.0,4.0,2/9/2025,Team Vitality,Team Liquid,Win,'3 : 2'
1,Less,1.14,227.0,20,17,5,3,78%,140.0,32%,0.0,1.0,-1.0,2/9/2025,Team Vitality,Team Liquid,Win,'3 : 2'
2,trexx,0.63,161.0,12,19,4,-7,70%,120.0,17%,1.0,4.0,-3.0,2/9/2025,Team Vitality,Team Liquid,Win,'3 : 2'
3,Kicks,0.62,103.0,9,16,7,-7,65%,66.0,25%,0.0,2.0,-2.0,2/9/2025,Team Vitality,Team Liquid,Win,'3 : 2'
4,Sayf,0.37,112.0,9,20,4,-11,52%,70.0,29%,2.0,1.0,1.0,2/9/2025,Team Vitality,Team Liquid,Win,'3 : 2'


In [225]:
matches_df.shape

(9300, 18)

In [226]:
matches_df.dtypes

,0
Player,object
R2.0,float64
ACS,float64
K,int64
D,int64
A,int64
+/- K/D,int64
KAST,object
ADR,float64
HS%,object


In [227]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Convert date column from object to datetime
matches_df["date"] = pd.to_datetime(matches_df["date"])

# Handle missing values
matches_df.fillna(0, inplace=True)

# Handle percentage columns like 'HS%' and 'KAST'
# Convert to numeric and handle any non-numeric values gracefully
matches_df['HS%'] = pd.to_numeric(matches_df['HS%'], errors='coerce')  # Convert to numeric, handle errors by coercing to NaN
matches_df['KAST'] = pd.to_numeric(matches_df['KAST'], errors='coerce')

# If these columns represent percentages, divide by 100 to scale them
matches_df['HS%'] = matches_df['HS%'] / 100
matches_df['KAST'] = matches_df['KAST'] / 100

# Encode categorical columns like 'Player', 'Team', 'vs Team'
label_encoder = LabelEncoder()
matches_df['Player'] = label_encoder.fit_transform(matches_df['Player'])
matches_df['Team'] = label_encoder.fit_transform(matches_df['Team'])
matches_df['vs Team'] = label_encoder.fit_transform(matches_df['vs Team'])

# Convert 'final_score' to two separate columns for individual scores
matches_df[['final_score_winner', 'final_score_loser']] = matches_df['final_score'].str.split(':', expand=True)

# Convert the split columns to numeric
matches_df['final_score_winner'] = pd.to_numeric(matches_df['final_score_winner'], errors='coerce')
matches_df['final_score_loser'] = pd.to_numeric(matches_df['final_score_loser'], errors='coerce')

# Drop 'final_score' column as it is now split into 'final_score_winner' and 'final_score_loser'
matches_df.drop(columns=['final_score'], inplace=True)

# Convert 'result' to integers (0 for Loss, 1 for Win)
matches_df['result'] = matches_df['result'].replace({'Loss': 0, 'Win': 1})

# Check the result
print(matches_df.dtypes)



Player                         int64
R2.0                         float64
ACS                          float64
K                              int64
D                              int64
A                              int64
+/- K/D                        int64
KAST                         float64
ADR                          float64
HS%                          float64
FK                           float64
FD                           float64
+/- FK/FD                    float64
date                  datetime64[ns]
Team                           int64
vs Team                        int64
result                         int64
final_score_winner           float64
final_score_loser            float64
dtype: object


<ipython-input-227-d8bea29d5d64>:35: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matches_df['result'] = matches_df['result'].replace({'Loss': 0, 'Win': 1})


In [228]:
from sklearn.ensemble import RandomForestClassifier
# maybe experiment with these values
rf = RandomForestClassifier(n_estimators=50, min_samples_split = 10, random_state = 1)

In [229]:
# Filter the dataset based on date
train = matches_df[matches_df["date"] < '2025-01-01']
test = matches_df[matches_df["date"] >= '2025-01-01']

# Convert 'date' column to numeric (days since the first date)
train['date'] = (train['date'] - train['date'].min()).dt.days
test['date'] = (test['date'] - test['date'].min()).dt.days


<ipython-input-229-11409d05edba>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['date'] = (train['date'] - train['date'].min()).dt.days
<ipython-input-229-11409d05edba>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['date'] = (test['date'] - test['date'].min()).dt.days


In [230]:
# test =  matches_df[matches_df["date"] > '2025-01-01']

In [231]:
predictors = ['K', 'D', 'A', '+/- K/D', 'ADR', 'HS%', 'FK', 'FD', 'KAST', 'Team', 'vs Team', 'date']

In [232]:
print(test[predictors].dtypes)


K            int64
D            int64
A            int64
+/- K/D      int64
ADR        float64
HS%        float64
FK         float64
FD         float64
KAST       float64
Team         int64
vs Team      int64
date         int64
dtype: object


In [233]:
rf.fit(train[predictors], train["result"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [234]:
# Now fit the model and predict
rf.fit(train[predictors], train["result"])
preds = rf.predict(test[predictors])

In [235]:
from sklearn.metrics import accuracy_score

In [236]:
acc = accuracy_score(test["result"], preds)

In [237]:
acc

0.48522727272727273

In [238]:
# Lets try to implement rolling averages!

In [239]:
combined = pd.DataFrame(dict(actual=test["result"], predicted=preds))
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1
actual,,
0,714,606
1,753,567


In [240]:
from sklearn.metrics import precision_score

In [241]:
precision_score(test["result"], preds)

0.4833759590792839

In [242]:
grouped_matches = matches.groupby("Team")

In [243]:
group = grouped_matches.get_group("Team Liquid")

In [244]:
group

,Players,R2.0,ACS,K,D,A,+/- K/D,KAST,ADR,HS%,FK,FD,+/- FK/FD,date,Team,vs Team
5,paTiTek,1.45,238.0,20,11,5,9,91%,155.0,26%,5.0,0.0,5.0,2025-02-09,Team Liquid,Team Vitality
6,Keiko,1.43,311.0,25,16,5,9,83%,200.0,22%,2.0,1.0,1.0,2025-02-09,Team Liquid,Team Vitality
7,kamyk,1.20,216.0,19,13,6,6,78%,130.0,32%,1.0,1.0,0.0,2025-02-09,Team Liquid,Team Vitality
8,kamo,0.85,171.0,13,15,5,-2,74%,109.0,24%,3.0,6.0,-3.0,2025-02-09,Team Liquid,Team Vitality
9,nAts,0.68,158.0,14,15,5,-1,70%,105.0,29%,1.0,3.0,-2.0,2025-02-09,Team Liquid,Team Vitality
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9150,Redgar,1.40,309.0,20,15,6,5,76%,202.0,31%,5.0,3.0,2.0,2023-02-22,Team Liquid,Team Secret
9151,Sayf,1.07,220.0,18,14,3,4,76%,151.0,30%,2.0,0.0,2.0,2023-02-22,Team Liquid,Team Secret
9152,Jamppi,0.97,234.0,15,17,7,-2,86%,143.0,17%,4.0,2.0,2.0,2023-02-22,Team Liquid,Team Secret
9153,nAts,0.95,176.0,14,16,10,-2,76%,108.0,32%,0.0,2.0,-2.0,2023-02-22,Team Liquid,Team Secret


In [258]:
# Convert 'HS%' column from percentage string to float (ensure all values are cleaned)
group['HS%'] = group['HS%'].apply(lambda x: str(x).replace('%', '') if isinstance(x, str) else x)
group['HS%'] = pd.to_numeric(group['HS%'], errors='coerce') / 100.0  # Convert to numeric and divide by 100

# Ensure all specified columns are numeric (including any that might have been left as strings)
cols_to_convert = ['K', 'D', 'A', '+/- K/D', 'ADR', 'HS%', 'FK', 'FD', 'KAST']
group[cols_to_convert] = group[cols_to_convert].apply(pd.to_numeric, errors='coerce')

# Now you can compute rolling averages
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")  # Ensure sorted by date
    rolling_stats = group[cols].rolling(3, closed='left').mean()  # Compute rolling mean
    group[new_cols] = rolling_stats  # Add the new columns to the group
    # group = group.dropna(subset=new_cols)  # Drop rows with NaN in the new columns
    return group

# Create new column names for rolling averages
new_cols = [f"{col}_rolling" for col in cols_to_convert]

# Apply rolling averages to the group
rolling_averages(group, cols_to_convert, new_cols)

# Apply rolling averages to each team
matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, cols_to_convert, new_cols))


<ipython-input-258-4ae81c9c9931>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['HS%'] = group['HS%'].apply(lambda x: str(x).replace('%', '') if isinstance(x, str) else x)
<ipython-input-258-4ae81c9c9931>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['HS%'] = pd.to_numeric(group['HS%'], errors='coerce') / 100.0  # Convert to numeric and divide by 100
<ipython-input-258-4ae81c9c9931>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

DataError: Cannot aggregate non-numeric type: object

DataError: Cannot aggregate non-numeric type: object